In [ ]:
import pandas as pd
import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [ ]:
print("Reading dataframe from train file")

In [ ]:
df =  pd.read_csv('train.csv')

In [ ]:
print("Read into dataframe from train file")

In [ ]:
def convert(date_time):
    format1 = '%m/%d/%Y %H:%M'
    dat = pd.to_datetime(date_time, format=format1).dt
    return dat.month, dat.day, dat.hour, dat.minute
   

In [ ]:
df['month'], df['day'], df['hour'], df['minute'] = convert(df['Ob'])

In [ ]:
print("Date column Converted")

In [ ]:
qc0 = {"R0","R0Z0","R0I0","R0I0Z0","R0B0","R0B0Z0","R0B1Z0","R1Z0","R1I0","R1B0","R1B0Z0","R1B1Z0","R2B0Z0","Z0","Z2","I0","B0","B0Z0","B0Z2","B1Z0"}
qc1 = {"R0Z2","R0I1","R0I0Z0","R0B1","R0B2Z0","R0B3Z0","R1","R1B0Z4","R1I2","R1B1","R1B2Z0","R2Z0","R2B0","R2B0Z2","R2B1Z0","R2B2Z0","R3B0Z0","R3B1Z0","I1","B0Z4","B1","B1Z2","B2Z0"}
qc2 = {"R0I2","R0I2Z0","R0I4","R0I4Z0","R0B0Z4","R0B1Z4","R0B2","R0B2Z4","R0B3","R0B3Z2","R1Z4","R1I4","R1B1Z4","R1B2","R1B2Z4","R1B3Z0","R2","R2Z2","R2Z4","R2B0Z4","R2B1Z2","R2B1Z4","R2B1","R2B2","R2B2Z2","R2B3Z0","R3","R3Z0","R3Z2","R3","R3I0","R3B0","R3B0Z2","R3B1","R3B1Z2","Z4","I2","B1Z4","B2","B2Z2","B2Z4","B3","B3Z0","B3Z2","B4","B4Z0","B4Z2"}
qc3 = {"R0Z4","R0I1Z4","R0I2Z4","R0I4Z4","R0B3Z4","R0B4Z0","R0B4Z2","R0B4Z4","R0B5Z0","R0B5Z2","R0B5Z4","R1B3","R1B3Z4","R1B4Z0","R1B4Z2","R1B4Z4","R1B5Z0","R1B5Z2","R1B5Z4","R2B2Z4","R2B3","R2B3Z2","R2B3Z4","R2B4Z0","R2B4Z2","R2B4Z4","R2B5Z0","R2B5Z2","R2B5Z4","R3Z4","R3I4","R3B0Z4","R3B1Z4","R3B2","R3B2Z0","R3B2Z2","R3B2Z4","R3B3","R3B3Z0","R3B3Z2","R3B3Z4","R3B4","R3B4Z0","R3B4Z2","R3B4Z4","R3B5","R3B5Z0","R3B5Z2","R3B5Z4","I4","B3Z4","B4Z4","B5","B5Z0","B5Z2","B5Z4"}

In [ ]:
df['qc_score'] = [-1]*(df.shape[0])

In [ ]:
def qc_score_flag(df_conv):
    for i in tqdm(range(df_conv.shape[0])):
        qc_flag = ""
        if(df['R_flag'][i] != -1):
            qc_flag += 'R'+str(df_conv['R_flag'][i])
        if(df['B_flag'][i] != -1):
            qc_flag += 'B'+str(df_conv['B_flag'][i])
        if(df['I_flag'][i] != -1):
            qc_flag += 'I'+str(df_conv['I_flag'][i])
        if(df['Z_flag'][i] != -1):
            qc_flag += 'Z'+str(df_conv['Z_flag'][i])

        qc_score = -1
        if(qc_flag.startswith("R4")):
            qc_score=3
        elif(qc_flag in qc0):
            qc_score = 0
        elif(qc_flag in qc1):
            qc_score = 1
        elif(qc_flag in qc2):
            qc_score = 2
        elif(qc_flag in qc3):
            qc_score = 3
            
        df_conv['qc_score'][i] = qc_score
    return df_conv

In [ ]:
df = qc_score_flag(df)

In [ ]:
df.groupby(['qc_score','target']).size()

In [ ]:
scalar_map = {}
for i,j in df.groupby('measure')['value']:
    scaler = StandardScaler()
    scaler.fit(j.values.reshape(-1,1))
    scalar_map[i] = {}
    scalar_map[i]["mean"] = scaler.mean_[0]
    scalar_map[i]["std"] = scaler.scale_[0]

In [ ]:
print(scalar_map)

In [ ]:
df['value'] = df.groupby('measure')['value'].apply(lambda x : ((((x - scalar_map[x.name].get("mean")) / scalar_map[x.name].get("std")) if x.name in scalar_map else x)))

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df[['measure', 'Station']])
transformed_data = enc.transform(df[['measure', 'Station']])
df_cat = pd.DataFrame.sparse.from_spmatrix(transformed_data, index=df.index,columns=enc.get_feature_names_out())

In [ ]:
df = df.drop(columns=['measure','Station'])

In [ ]:
df = pd.concat([df, df_cat], axis=1)

In [ ]:
df = df.drop(columns=['Ob'])

In [ ]:
print("DF Shape after One hot encoding and Normalisation : ", df.shape)

In [ ]:
df.to_csv("Train_updated.csv", index=False)

In [ ]:
df_test =  pd.read_csv('test.csv')

In [ ]:
transformed_test = enc.transform(df_test[['measure', 'Station']])

In [ ]:
df_test['value'] = df_test.groupby('measure')['value'].apply(lambda x : ((((x - scalar_map[x.name].get("mean")) / scalar_map[x.name].get("std")) if x.name in scalar_map else x)))

In [ ]:
df_test['month'], df_test['day'], df_test['hour'], df_test['minute'] = convert(df_test['Ob'])

In [ ]:
df_test['qc_score'] = [-1]*(df_test.shape[0])

In [ ]:
df_test = qc_score_flag(df_test)

In [ ]:
df_test.groupby(['qc_score']).size()

In [ ]:
df_test.columns

In [ ]:
df_test = df_test.drop(columns=['Ob','measure','Station'], axis=1)

In [ ]:
df_test_cat = pd.DataFrame.sparse.from_spmatrix(transformed_test, index=df_test.index,columns=enc.get_feature_names())

In [ ]:
X_test = df_test = pd.concat([df_test, df_test_cat], axis=1)

In [ ]:
print("DF Shape before writing into CSV for test : ", df_test.shape)

In [ ]:
X_test.to_csv("Test_updated.csv", index=False)